# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
from sklearn.preprocessing import Normalizer
X_norm=pd.DataFrame(Normalizer().fit_transform(X))

In [6]:
# train/test split
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

models = {'rl': LogisticRegression(),
          'dt': DecisionTreeClassifier()}

In [8]:
for model in models.keys():
    models[model].fit(X_train, y_train)

In [9]:
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)
y_pred.head()

,rl,dt
1483,1,0
2185,1,0
2520,1,1
3721,1,0
3727,0,0


In [10]:
# Evaluate each model
from sklearn.metrics import mean_squared_error
from sklearn import metrics
for model in models.keys():
    print(model)
    print('MSE:',np.sqrt(metrics.mean_squared_error(y_pred[model], y_test)))
    print('f1-score:',metrics.f1_score(y_pred[model], y_test), 'ACC:',metrics.accuracy_score(y_pred[model], y_test))

rl
MSE: 0.6148170459575759
f1-score: 0.6059763724808894 ACC: 0.622
dt
MSE: 0.6792152334373349
f1-score: 0.5386666666666666 ACC: 0.5386666666666666


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [11]:
n_estimators = 100
# set a seed for reproducibility
np.random.seed(123)
n_samples = X_train.shape[0]
# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

In [12]:
np.random.seed(123)
seeds = np.random.randint(1, 10000, size=n_estimators)
trees = {}
for i in range(n_estimators):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
# Predict 
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = trees[i].predict(X_test)

In [13]:
np.random.seed(123)
seeds = np.random.randint(1, 10000, size=n_estimators)
trees2 = {}
for i in range(n_estimators):
    trees2[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=2, random_state=seeds[i])
    trees2[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
# Predict 
y_pred_df2 = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df2.iloc[:, i] = trees2[i].predict(X_test)

In [14]:
np.random.seed(123)
seeds = np.random.randint(1, 10000, size=n_estimators)
reglog = {}
for i in range(n_estimators):
    reglog[i] = LogisticRegression(random_state=seeds[i])
    reglog[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
# Predict 
y_pred_df3 = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df3.iloc[:, i] = trees[i].predict(X_test)

# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [15]:
Mtrs=pd.DataFrame(columns=['Model','RMSE','ACC',"F1"],data=[])
Mtrs.shape[0]
y_pred = pd.DataFrame(index=X_test.index, columns=list(range(300)))
for i in range(0,100):
    y_pred.iloc[:, i] = y_pred_df[i]
for i in range(100,200):
    y_pred.iloc[:, i] = y_pred_df2[i-100]
for i in range(200,300):
    y_pred.iloc[:, i] = y_pred_df3[i-200]
y_pred2 = (y_pred.sum(axis=1) >= (300 / 2)).astype(np.int)
RMSE=np.sqrt(metrics.mean_squared_error(y_pred2, y_test))
ACC=metrics.accuracy_score(y_pred2, y_test)
F1=metrics.f1_score(y_pred2, y_test)
Mtrs.loc[0] = ['Majority Voting',RMSE,ACC,F1]
Mtrs

,Model,RMSE,ACC,F1
0,Majority Voting,0.583095,0.66,0.662252


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [16]:
t = range(0,300,5)
j=1
for i in t:
    y_pred3 = (y_pred.sum(axis=1) >= i).astype(np.int)
    RMSE=np.sqrt(metrics.mean_squared_error(y_pred3, y_test))
    ACC=metrics.accuracy_score(y_pred3, y_test)
    F1=metrics.f1_score(y_pred3, y_test)
    Mtrs.loc[j] = [ "MV tresh: %.2f%s"%((i/300)*100, '%'),RMSE,ACC,F1]
    j+=1
#select the one that maximizes the F1-Score
Mtrs[Mtrs['F1']==np.max(Mtrs['F1'])]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


,Model,RMSE,ACC,F1
18,MV tresh: 28.33%,0.6159,0.620667,0.713927


In [17]:
#select the one that maximizes the Accuracy
Mtrs[Mtrs['ACC']==np.max(Mtrs['ACC'])]

,Model,RMSE,ACC,F1
29,MV tresh: 46.67%,0.57908,0.664667,0.681847


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [18]:
samples_oob = []
# show the "out-of-bag" observations for each sample
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

In [19]:
errors1 = np.zeros(n_estimators)
for i in range(n_estimators):
    y_pred_df4 = trees[i].predict(X_train.iloc[samples_oob[i]])
    errors1[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_df4)

In [20]:
errors2 = np.zeros(n_estimators)
for i in range(n_estimators):
    y_pred_df5 = trees2[i].predict(X_train.iloc[samples_oob[i]])
    errors2[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_df5)

In [21]:
errors3 = np.zeros(n_estimators)
for i in range(n_estimators):
    y_pred_df6 = reglog[i].predict(X_train.iloc[samples_oob[i]])
    errors3[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_df6)

In [22]:
errors=np.concatenate((errors1, errors2,errors3))

In [24]:
Mtrs2=pd.DataFrame(columns=['Model','RMSE','ACC',"F1"],data=[])
Mtrs2.shape[0]
alpha = (1 - errors) / (1 - errors).sum()
weighted_sum_1 = ((y_pred) * alpha).sum(axis=1)
y_pred4 = (weighted_sum_1 >= 0.5).astype(np.int)
RMSE=np.sqrt(metrics.mean_squared_error(y_pred4, y_test))
ACC=metrics.accuracy_score(y_pred4, y_test)
F1=metrics.f1_score(y_pred4, y_test)
Mtrs2.loc[0] = ['weighted voting',RMSE,ACC,F1]
Mtrs2

,Model,RMSE,ACC,F1
0,weighted voting,0.584237,0.658667,0.659121


# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [33]:
t = range(0,100,5)
j=1
for i in t:
    y_pred5 = (weighted_sum_1 >= i/100).astype(np.int)
    RMSE=np.sqrt(metrics.mean_squared_error(y_pred5, y_test))
    ACC=metrics.accuracy_score(y_pred5, y_test)
    F1=metrics.f1_score(y_pred5, y_test)
    Mtrs2.loc[j] = [ "WV prob: %.0f%s"%(i, '%'),RMSE,ACC,F1]
    j+=1
#select the one that maximizes the F1-Score
Mtrs2[Mtrs2['F1']==np.max(Mtrs2['F1'])]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


,Model,RMSE,ACC,F1
7,WV prob: 30%,0.609918,0.628,0.711777


In [34]:
#select the one that maximizes the Accuracy
Mtrs2[Mtrs2['ACC']==np.max(Mtrs2['ACC'])]

,Model,RMSE,ACC,F1
0,weighted voting,0.584237,0.658667,0.659121
10,WV prob: 45%,0.584237,0.658667,0.680798
11,WV prob: 50%,0.584237,0.658667,0.659121


# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [52]:
X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(300)))

for i in range(n_estimators):
    X_train_2[i] = trees[i].predict(X_train)
for i in range(100,200):
    X_train_2.iloc[:, i] = trees2[i-100].predict(X_train)
for i in range(200,300):
    X_train_2.iloc[:, i] = reglog[i-200].predict(X_train)

In [54]:
Mtrs3=pd.DataFrame(columns=['Model','RMSE','ACC',"F1"],data=[])
Mtrs3.shape[0]
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv = 5 )
lr.fit(X_train_2, y_train)
y_pred6 = lr.predict(y_pred)
RMSE=np.sqrt(metrics.mean_squared_error(y_pred6, y_test))
ACC=metrics.accuracy_score(y_pred6, y_test)
F1=metrics.f1_score(y_pred6, y_test)
Mtrs3.loc[0] = ['LogRegCV',RMSE,ACC,F1]
Mtrs3

,Model,RMSE,ACC,F1
0,LogRegCV,0.585947,0.656667,0.660514


In [55]:
y_pred6

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [61]:
t = range(0,100,5)
j=1
for i in t:
    y_pred7 = np.where(lr.predict_proba(y_pred)[:,1] >= i/100, 1, 0)
    RMSE=np.sqrt(metrics.mean_squared_error(y_pred7, y_test))
    ACC=metrics.accuracy_score(y_pred7, y_test)
    F1=metrics.f1_score(y_pred7, y_test)
    Mtrs3.loc[j] = [ "LogReg tresh: %.0f%s"%(i, '%'),RMSE,ACC,F1]
    j+=1
#select the one that maximizes the F1-Score
Mtrs3[Mtrs3['F1']==np.max(Mtrs3['F1'])]

,Model,RMSE,ACC,F1
5,LogReg tresh: 20%,0.621825,0.613333,0.70971


In [62]:
#select the one that maximizes the Accuracy
Mtrs3[Mtrs3['ACC']==np.max(Mtrs3['ACC'])]

,Model,RMSE,ACC,F1
0,LogRegCV,0.585947,0.656667,0.660514
11,LogReg tresh: 50%,0.585947,0.656667,0.660514
